In [9]:
import sys, os
from datetime import datetime, timedelta

project_root = os.path.abspath("..")
sys.path.insert(0, project_root)

data_dir = os.path.join(project_root, "data")
os.makedirs(data_dir, exist_ok=True)

print("project_root:", project_root)
print("data_dir:", data_dir)


project_root: /Users/mathisvillaret/Documents (mac)/Le CODE
data_dir: /Users/mathisvillaret/Documents (mac)/Le CODE/data


In [ ]:
from import_other_options import import_sp500_options_data

start_date = datetime.now().date()
end_date = (datetime.now() + timedelta(days=360)).date()

spx_df = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir
)

print("✓ spx_df loaded:", len(spx_df))
spx_df.head()


⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
    Expected path should look like: .../Option_Pricing/data/
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251212_005018.csv
✓ spx_df loaded: 15197


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,best_bid_price,best_ask_price,volume,instrument_name,contractSymbol,underlying_ticker
0,2025-12-12,2025-12-12,2200.0,C,7.796509,4701.98,6901.0,21.0,4714.2,4726.70,4714.2,4726.70,4.0,^SPX-2025-12-12-2200.0-C,SPXW251212C02200000,^SPX
1,2025-12-12,2025-12-12,2200.0,P,4.125005,0.05,6901.0,6.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-12-2200.0-P,SPXW251212P02200000,^SPX
2,2025-12-12,2025-12-12,2400.0,P,3.812500,0.10,6901.0,10.0,0.0,0.05,0.0,0.05,10.0,^SPX-2025-12-12-2400.0-P,SPXW251212P02400000,^SPX
3,2025-12-12,2025-12-12,2600.0,P,3.531251,0.05,6901.0,87.0,0.0,0.05,0.0,0.05,23.0,^SPX-2025-12-12-2600.0-P,SPXW251212P02600000,^SPX
4,2025-12-12,2025-12-12,2800.0,P,3.281252,0.12,6901.0,251.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-12-2800.0-P,SPXW251212P02800000,^SPX


In [11]:
from iv_surface_spx import SPXIVSurface, SurfaceConfig

cfg = SurfaceConfig(
    r=0.05,
    min_bid=0.01,
    max_rel_spread=0.25,
    min_oi=10,
    min_volume=1,
    grid_n=60,
    rbf_smoothing=0.5
)

spx_surface = SPXIVSurface(spx_df, cfg)
print("Rows after cleaning:", len(spx_surface.df))
spx_surface.df.head()


Rows after cleaning: 6708


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,...,underlying_ticker,T,S,bid,ask,mid,rel_spread,iv_pct,F,x
0,2025-12-12,2025-12-12,6765.0,P,0.124765,0.40,6901.0,930,0.30,0.35,...,^SPX,0.000001,6901.0,0.30,0.35,0.325,0.153846,12.476461,6920.950003,-0.022791
1,2025-12-12,2025-12-12,6780.0,P,0.118966,0.55,6901.0,1621,0.40,0.50,...,^SPX,0.000001,6901.0,0.40,0.50,0.450,0.222222,11.896633,6920.950003,-0.020576
2,2025-12-12,2025-12-12,6800.0,P,0.109353,0.90,6901.0,2992,0.60,0.75,...,^SPX,0.000001,6901.0,0.60,0.75,0.675,0.222222,10.935339,6920.950003,-0.017630
3,2025-12-12,2025-12-12,6805.0,P,0.106088,1.00,6901.0,499,0.65,0.80,...,^SPX,0.000001,6901.0,0.65,0.80,0.725,0.206897,10.608804,6920.950003,-0.016895
4,2025-12-12,2025-12-12,6810.0,P,0.103799,1.15,6901.0,1062,0.80,0.90,...,^SPX,0.000001,6901.0,0.80,0.90,0.850,0.117647,10.379925,6920.950003,-0.016161


In [12]:
fig = spx_surface.plot(
    title="SPX Implied Volatility Surface (OTM, forward-moneyness)",
    interpolate=True
)
fig.show()


### Summary (SPX IV Surface — OTM, forward-moneyness)

- The surface exhibits a **strong downside skew**: implied volatility increases sharply as  
  **log-moneyness** $\ln(K/F)$ becomes more negative (deep OTM puts).  
  This is typical for SPX and reflects **crash-risk insurance demand**.

- Volatility varies with **time to expiry**, and the **term structure depends on moneyness**  
  (the maturity effect is not uniform across strikes).

- The “striped” pattern in the white points is expected because option quotes exist on  
  **discrete expiries and strikes**; the smooth surface is an interpolation across that grid.

- Extremely high IV levels (e.g., **60–70%**) in the far left tail are likely driven by  
  **illiquid/unstable quotes and interpolation extrapolation**, so the outer tail should be  
  treated cautiously (e.g., **outlier filtering** or **restricting the moneyness range** before fitting).
